In [4]:
%pip install boto3


   ---------------------------------------- 0.0/13.9 MB ? eta -:--:--
   -------------------------- ------------- 9.2/13.9 MB 47.4 MB/s eta 0:00:01
   ---------------------------------------- 13.9/13.9 MB 45.9 MB/s eta 0:00:00

  Attempting uninstall: urllib3

    Found existing installation: urllib3 2.5.0

    Uninstalling urllib3-2.5.0:

      Successfully uninstalled urllib3-2.5.0

   ---------------------------------------- 0/5 [urllib3]
   ---------------- ----------------------- 2/5 [botocore]
   ---------------- ----------------------- 2/5 [botocore]
   ---------------- ----------------------- 2/5 [botocore]
   ---------------- ----------------------- 2/5 [botocore]
   ---------------- ----------------------- 2/5 [botocore]
   ---------------- ----------------------- 2/5 [botocore]
   ---------------- ----------------------- 2/5 [botocore]
   ---------------- ----------------------- 2/5 [botocore]
   ---------------- ----------------------- 2/5 [botocore]
   ---------------- ---

In [ ]:
import boto3
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, FloatType

spark = SparkSession.builder.appName("S3 iris").getOrCreate()
sc = spark.sparkContext

# get the iris data from the specific s3 bucket
s3 = boto3.client("s3")
bucket_name = "assignment14irisdatasets"
file_name = "iris.data"
s3.download_file(bucket_name, file_name, file_name)

# the schema of the iris data
schema = StructType([
    StructField("sepal_length", FloatType(), True),
    StructField("sepal_width", FloatType(), True),
    StructField("petal_length", FloatType(), True),
    StructField("petal_width", FloatType(), True),
    StructField("class", StringType(), True),
])

# read the iris data from the specific s3 bucket as dataframe
df = spark.read.csv(file_name, header = False, schema = schema)

# turn the dataframe into a temporary table
df.createOrReplaceTempView("iris")

# print some data to verify the data is read correctly
print(df.show())

+------------+-----------+------------+-----------+-----------+
|sepal_length|sepal_width|petal_length|petal_width|      class|
+------------+-----------+------------+-----------+-----------+
|         5.1|        3.5|         1.4|        0.2|Iris-setosa|
|         4.9|        3.0|         1.4|        0.2|Iris-setosa|
|         4.7|        3.2|         1.3|        0.2|Iris-setosa|
|         4.6|        3.1|         1.5|        0.2|Iris-setosa|
|         5.0|        3.6|         1.4|        0.2|Iris-setosa|
|         5.4|        3.9|         1.7|        0.4|Iris-setosa|
|         4.6|        3.4|         1.4|        0.3|Iris-setosa|
|         5.0|        3.4|         1.5|        0.2|Iris-setosa|
|         4.4|        2.9|         1.4|        0.2|Iris-setosa|
|         4.9|        3.1|         1.5|        0.1|Iris-setosa|
|         5.4|        3.7|         1.5|        0.2|Iris-setosa|
|         4.8|        3.4|         1.6|        0.2|Iris-setosa|
|         4.8|        3.0|         1.4| 

In [2]:
# run the SQL search
query = """SELECT class, COUNT(*) as count
           FROM iris 
           GROUP BY class"""
result = spark.sql(query)
result.show()

+---------------+-----+
|          class|count|
+---------------+-----+
| Iris-virginica|   50|
|    Iris-setosa|   50|
|Iris-versicolor|   50|
+---------------+-----+



In [3]:
#2. find the average sepal length of each class
query = """
        SELECT class, AVG(petal_length) as avg_petal_length
        FROM iris
        GROUP BY class
        """
result = spark.sql(query)
result.show()

+---------------+------------------+
|          class|  avg_petal_length|
+---------------+------------------+
| Iris-virginica| 5.551999988555909|
|    Iris-setosa|1.4639999961853027|
|Iris-versicolor| 4.259999980926514|
+---------------+------------------+



In [4]:
#3. find the species with maximum average sepal width
# calculate the average sepal width for each class and order by descending order, then select the top 1
query = """
        SELECT class, AVG(sepal_width) as avg_sepal_width
        FROM iris
        GROUP BY class
        ORDER BY avg_sepal_width DESC
        LIMIT 1
        """
result = spark.sql(query)
result.show()


+-----------+----------------+
|      class| avg_sepal_width|
+-----------+----------------+
|Iris-setosa|3.41800000667572|
+-----------+----------------+

